In [ ]:
import os
import sys
# os.chdir("../")
sys.path.append('../tsp_nls')
sys.path.append('../tsp')
sys.path.append("..")  # This points to `DeepACO`

TSPLIB_DIR = "../data/tsp/TSPLIB"
tsp_files = [f for f in os.listdir(TSPLIB_DIR) if f.endswith(".tsp")]
print(tsp_files)

In [ ]:
print("Current Working Directory:", os.getcwd())
import torch
from torch.distributions import Categorical, kl
# from d2l.torch import Animator

from tsp_nls.greedy import test_greedy, GreedyTSP

torch.manual_seed(12345)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Test Performance of DeepACO

## TSP500

In [ ]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 500
n_ants = 50
t_aco = [2, 10]
k_sparse = 50
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=500

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
    test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

## TSP1000

In [ ]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 1000
n_ants = 100
t_aco = [2, 10]
k_sparse = 100
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=1000

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
    test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))
    
# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

# Test Generalization performance of DeepACO across TSP scales

## Test TSP500

In [ ]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=500

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
    test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))


# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'generalization_cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

## Test TSP1000

In [ ]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=1000

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

res, avg_ga_best, duration_ga = genetic.test_genetic(
    test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
print(res)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))


# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'generalization_cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

## Test TSPLIB

In [ ]:
from tsp_nls.utils import load_TSPLIB_test_instance
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'

TSPLIB_DIR = '../data/tsp/TSPLIB'
file_names=['bier127.tsp', 'ch130.tsp', 'ch150.tsp', 'eil101.tsp']
tsplib_test_list = load_TSPLIB_test_instance(dir=TSPLIB_DIR, file_names=file_names, device=device, start_node=0, k_sparse = k_sparse, normalized=True)
res_avg_cost = {}

    
# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(tsplib_test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(tsplib_test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(tsplib_test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(tsplib_test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[10] # limited runtime

res, avg_ga_best, duration_ga = genetic.test_genetic(
    tsplib_test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
print(res)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

import csv
RES_DIR = 'results/'
f_out = 'generalization_cost_tsplib100.csv'
test_set = 'TSPLIB100'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])